In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm

path = '/nfs/misc/literature/shyama/OTAR_Old_Pipeline/FullTextJan2020/'

In [2]:
all_files = sorted(glob.glob(path+'Annot_PMC*.xml'))

In [3]:
len(all_files)

266

In [4]:
all_files[0]

'/nfs/misc/literature/shyama/OTAR_Old_Pipeline/FullTextJan2020/Annot_PMC1240624_PMC1474480.xml'

In [4]:
def getfileblocks(file_path):

    subFileBlocks = []

    with open(file_path, 'r') as fh:
        for line in fh:
            if line.startswith('<!DOCTYPE'):
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line
                
    return subFileBlocks

In [5]:
def GP_soup(file_soup):
    dict_sentences = defaultdict(list)
    for each_ztag in file_soup.find_all('z:uniprot'):
        try:
            gene_sentence = each_ztag.findParents('plain')[0].text
            dict_sentences[gene_sentence].append(each_ztag) 
        except:
            pass
    return dict_sentences


from fuzzywuzzy import fuzz 
def compare_with_ml_annotations(r, z_tags, ml_json):
    FP_removed_z_tags = []
    for each_z_tag in z_tags:
        for each_ml_annotation in r.json()['annotations']:
            score = fuzz.token_set_ratio(each_ml_annotation[3],each_z_tag.text)
            if  score ==100 and each_ml_annotation[2]=='GP':
                FP_removed_z_tags.append(each_z_tag)
    
    return list(set(z_tags) - set(FP_removed_z_tags))



In [6]:
import requests

def get_unique_fp_tags(GP_dict_sentences):
    url = 'http://ai-capo-api-1:5200/spaCy_ner_predictor?text_sentence='

    FP_dict_sentences =defaultdict(list)
    set_FPs = set()
    
    for each_uniprot_sentence, z_tags in GP_dict_sentences.items():
        r = requests.get(url+each_uniprot_sentence)
        if r.status_code == 200 and r.json()['status']==200:
            FPs = compare_with_ml_annotations(r, z_tags,r.json())
            if FPs:
                FP_dict_sentences[each_uniprot_sentence].append(FPs)
        else:
            print(each_uniprot_sentence)
        
    for FP_sent,FP_tag in FP_dict_sentences.items():
        for each_ztag in FP_tag[0]:
            set_FPs.add(each_ztag)

    return list(set_FPs)
        

In [7]:
path_new = '/nfs/misc/literature/Santosh_Tirunagari/OTAR_dumps/FP_removed/'
# for each_file_content in subFileBlocks:
for each_file_path in all_files:
    ss = getfileblocks(each_file_path)
    for each_article in tqdm(ss):
        soup = BeautifulSoup(each_article, 'lxml')
        GP_dict_sentences = GP_soup(soup)
        if GP_dict_sentences:
            unique_fps = get_unique_fp_tags(GP_dict_sentences)

            for each_fp in unique_fps:
                for tag in soup.findAll('z:uniprot', attrs={'ids':each_fp['ids']}):
                    tag.unwrap()

            new_file_content = str(soup).replace('<!DOCTYPE >\n<html><body><p>JATS-archivearticle1.dtd"&gt;','<!DOCTYPE "JATS-archivearticle1.dtd">').replace('</p></body></html>','')

            with open(path_new+'edited_'+each_file_path.split('/')[-1], 'at') as f:
                f.write(new_file_content)



 26%|██▌       | 2607/10000 [08:27<1:38:28,  1.25it/s]

Australia: F Alard, J Alcoe, F Alford, C Allan, J Amerena, R Anderson, N Arnold, T Arsov, D Ashby, C Atkinson, L Badhni, M Balme, D Barton, B Batrouney, C Beare, T Beattie, J Beggs, C Bendall, C Bendall, A Benz, A Bond, R Bradfield, J Bradshaw, S Brearley, D Bruce, J Burgess, J Butler, M Callary, J Campbell, K Chambers, J Chow, S Chow, K Ciszek, P Clifton, P Clifton-Bligh, V Clowes, P Coates, C Cocks, S Cole, D Colquhoun, M Correcha, B Costa, S Coverdale, M Croft, J Crowe, S Dal Sasso, W Davis, J Dunn, S Edwards, R Elder, S El-Kaissi, L Emery, M England, O Farouque, M Fernandez, B Fitzpatrick, N Francis, P Freeman, A Fuller, D Gale, V Gaylard, C Gillzan, C Glatthaar, J Goddard, V Grange, T Greenaway, J Griffin, A Grogan, S Guha, J Gustafson, P S Hamblin, T Hannay, C Hardie, A Harper, G Hartl, A Harvey, S Havlin, K Haworth, P Hay, L Hay, B Heenan, R Hesketh, A Heyworth, M Hines, G Hockings, A Hodge, L Hoffman, L Hoskin, M Howells, D Hunt, A Hunt, W Inder, W Inder, D Jackson, A Jovanovsk

 29%|██▊       | 2872/10000 [10:42<52:19,  2.27it/s]  

"acinic cell carcinoma of the lung, adenocarcinoma and large cell carcinoma of lung, adenomatous hyperplasia of the rete testis, adenosarcoma of the renal pelvis with heterologous elements, adenosarcoma of the uterus with sarcomatous overgrowth, adrenal cortical adenoma in the spinal canal, aggressive digital papillary adenoma and adenocarcinoma, anaplastic carcinoma of the lung, atypical burkitt lymphoma arising from follicular lymphoma, atypical duct hyperplasia of the breast, atypical fibrous histiocytoma of the skin, atypical polypoid adenomyoma of the uterus, b-cell lymphoma of mucosa-associated lymphoid tissue, benign cystic mesothelioma of the liver, benign epithelioid peripheral nerve sheath tumor of the soft tissues, benign mesenchymal tumor of lymph node, benign mesenchymal tumor of the uterus, benign schwannoma of the digestive tract, bile duct adenoma clear cell type, bizarre parosteal osteochondromatous proliferation of bone, borderline epithelial lesions of the breast, bo

 29%|██▉       | 2923/10000 [11:09<1:08:17,  1.73it/s]

...

# hash tables (lookups) 
# entries per hash table 


 98%|█████████▊| 9758/10000 [57:02<01:47,  2.26it/s] 

# AMD 5  


  6%|▌         | 555/10000 [05:01<1:36:24,  1.63it/s]

KeyboardInterrupt: 

In [ ]:
# offset = 25

# start_index = key_fp.find(value_fp[0][0].text)
# end_index = start_index+len(value_fp[0][0].text)

# key_fp[start_index:end_index]

# exact = value_fp[0][0].text
# prefix = key_fp[max(0,start_index-offset):start_index-1]
# postfix = key_fp[end_index+1:end_index+offset]
            
# [prefix, exact, postfix]      

# # make sure the index starts from 0, if the start_index-offset<0, set it 0
# # the total offset incuding prefix and posfix is 50
# 

# value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation = 'http://purl.uniprot.org/uniprot/'+value_fp[0][0].attrs['ids'].split(',')[0]
# normalisation